# MNIST画像の分類 - Part2:モデルのデプロイ

Part1で作成したMNISTデータセットに対する画像の分類モデルをAzure Container InstancesにWebサービスとしてデプロイし、テストを実行します。
<br>
<br>
こちらは以下のチュートリアルをベースにしています。
<br>
https://docs.microsoft.com/ja-jp/azure/machine-learning/service/tutorial-deploy-models-with-aml

## 1．環境のセットアップ

必要なパッケージをインポートします。

In [ ]:
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import azureml
from azureml.core import Workspace, Run, Model

ワークスペースに接続し、モデルを取得します。

In [ ]:
ws = Workspace.from_config()
model = Model(ws, "sklearn_mnist")
model.download(target_dir=os.getcwd(), exist_ok=True)

# ダウンロードしたモデルの確認
file_path = os.path.join(os.getcwd(), "sklearn_mnist_model.pkl")
os.stat(file_path)

## 2．ローカルでのモデルのテスト

テストデータを読み込みます。

In [ ]:
from utils import load_data
import os

data_folder = os.path.join(os.getcwd(), "data")
X_test = load_data(os.path.join(data_folder, 'test-images.gz'), False) / 255.0
y_test = load_data(os.path.join(data_folder, 'test-labels.gz'), True).reshape(-1)

テストデータを使った推論を実行します。

In [ ]:
import pickle
from sklearn.externals import joblib

clf = joblib.load(os.path.join(os.getcwd(), "sklearn_mnist_model.pkl"))
y_hat = clf.predict(X_test)

推論結果に対して、混同行列を調査します。

In [ ]:
from sklearn.metrics import confusion_matrix

conf_mx = confusion_matrix(y_test, y_hat)
print(conf_mx)
print("Overall accuracy:", np.average(y_hat==y_test))

In [ ]:
# 混同行列をグラフで表示します。
row_sums = conf_mx.sum(axis=1, keepdims=True)
norm_conf_mx = conf_mx / row_sums
np.fill_diagonal(norm_conf_mx, 0)

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(111)
cax = ax.matshow(norm_conf_mx, cmap=plt.cm.bone)
ticks = np.arange(0, 10, 1)
ax.set_xticks(ticks)
ax.set_yticks(ticks)
ax.set_xticklabels(ticks)
ax.set_yticklabels(ticks)
fig.colorbar(cax)
plt.ylabel('true labels', fontsize=14)
plt.xlabel('predicted values', fontsize=14)
plt.savefig('conf.png')
plt.show()

## 3．Webサービスとしてデプロイ

Webサービスの呼び出しで実行されるスコアリングスクリプトを作成します。スコアリングスクリプトには、コンテナ起動時に実行されモデルを取得するinit()関数と、推論を実行するrun(input_data)を含める必要があります。

In [ ]:
%%writefile score.py
import json
import numpy as np
import os
import pickle
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression

from azureml.core.model import Model

def init():
    global model
    model_path = Model.get_model_path("sklearn_mnist")
    model = joblib.load(model_path)

def run(raw_data):
    data = np.array(json.loads(raw_data)["data"])
    y_hat = model.predict(data)
    return y_hat.tolist()    

環境ファイルを作成します。この環境ファイルでは、呼び出されるスクリプトのパッケージ依存関係を指定します。

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies

myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn")

with open("myenv.yml", "w") as f:
    f.write(myenv.serialize_to_string())

環境ファイルの内容を確認します。

In [ ]:
with open("myenv.yml", "r") as f:
    print(f.read())

コンテナの構成ファイルを作成します。

In [ ]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1,
                                              memory_gb=1,
                                              tags={"data": "MNIST", "method": "sklearn"},
                                              description="Predict MNIST with sklearn")

Azure Container Instanceにデプロイします。

In [ ]:
%%time
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage

# コンテナイメージの構成
image_config = ContainerImage.image_configuration(execution_script="score.py",
                                                 runtime="python",
                                                 conda_file="myenv.yml")

# コンテナのデプロイ
service = Webservice.deploy_from_model(workspace=ws,
                                      name="sklearn-mnist-svc",
                                      deployment_config=aciconfig,
                                      models=[model],
                                      image_config=image_config)

service.wait_for_deployment(show_output=True)

Webサービスのエンドポイントを確認します。

In [ ]:
print(service.scoring_uri)

## 4．デプロイされたサービスのテスト

SDKを使用して、サービスをテストします。

In [ ]:
import json

# テストデータから30画像をランダムで取り出し
n = 30
sample_indices = np.random.permutation(X_test.shape[0])[0:n]

test_samples = json.dumps({"data":X_test[sample_indices].tolist()})
test_sampels = bytes(test_samples, encoding="utf8")

# 推論の実行
service = Webservice(ws, "sklearn-mnist-svc")
result = service.run(input_data=test_samples)

# 実際の画像との比較
i = 0
plt.figure(figsize = (20, 1))

for s in sample_indices:
    plt.subplot(1, n, i + 1)
    plt.axhline('')
    plt.axvline('')
    
    font_color = 'red' if y_test[s] != result[i] else 'black'
    clr_map = plt.cm.gray if y_test[s] != result[i] else plt.cm.Greys
    
    plt.text(x=10, y =-10, s=result[i], fontsize=18, color=font_color)
    plt.imshow(X_test[s].reshape(28, 28), cmap=clr_map)
    
    i = i + 1
plt.show()

REST APIの呼び出しにより、サービスをテストします。

In [ ]:
import requests

random_index = np.random.randint(0, len(X_test)-1)
input_data = "{\"data\": [" + str(list(X_test[random_index])) + "]}"

headers = {'Content-Type':'application/json'}

resp = requests.post(service.scoring_uri, input_data, headers=headers)

print("POST to url", service.scoring_uri)
print("label:", y_test[random_index])
print("prediction:", resp.text)